Here, the performance of the lookup tables optimized in the "example_F_lookup_training_test.ipynb" on state stabilization for longer time durations can be tested.

In [49]:
# ruff: noqa
import sys, os
sys.path.append(os.path.abspath("./../feedback-grape"))
sys.path.append(os.path.abspath("./../"))

# ruff: noqa
from feedback_grape.utils.operators import identity
from feedback_grape.utils.states import coherent
import jax.numpy as jnp
import jax
from feedback_grape.utils.fidelity import ket2dm, fidelity

jax.config.update("jax_enable_x64", True)

# Test parameters
N_timesteps = 1000 # Number of time steps to test
results_dir = "./evaluation_results/"

# Physical parameters
N_cav = 30  # number of cavity modes
N_snap = 15

alpha = 2
psi_target = coherent(N_cav, alpha) + coherent(N_cav, -alpha)

# Normalize psi_target before constructing rho_target
psi_target = psi_target / jnp.linalg.norm(psi_target)

rho_target = ket2dm(psi_target)

c_ops = [jnp.sqrt(0.005) * destroy(N_cav)]*2 # Lindblad jump operators

In [50]:
from dynamiqs import mesolve
from tqdm import tqdm

# Propagate the state
fidelities = []
result = mesolve(
    H=identity(N_cav),
    jump_ops=c_ops,
    rho0=rho_target,
    tsave=jnp.linspace(0, N_timesteps+1, N_timesteps+1),
)

for rho in result.states:
    fidelities.append(fidelity(C_target=rho_target, U_final=rho.to_jax(), evo_type="density"))

# Reshape back to matrix form
jnp.savez(results_dir + "fidelities_no_feedback.npz", fidelities=fidelities)